# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. Natalia Lopez Gallego
2. Julian Monsalve Acevedo
3. Valentina Aristizabal
4. David Esteban De Los Rios
5. Santiago Gonzalez Quiroz

In [1]:
# Agrege acá el código para importar las librerias
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.functions import to_timestamp
from  pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import collect_list
from  pyspark.sql.functions import regexp_replace
from pyspark.sql.types import IntegerType

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [2]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

!hdfs dfs -copyFromLocal 'asistencia.csv' /tmp

In [3]:
!hdfs dfs -copyFromLocal 'evaluacion.csv' /tmp

In [4]:
!hadoop fs -ls /tmp

Found 2 items
-rw-r--r--   1 root supergroup      93783 2021-05-15 00:50 /tmp/asistencia.csv
-rw-r--r--   1 root supergroup     112572 2021-05-15 00:50 /tmp/evaluacion.csv


In [5]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...


Asistencia = ContextoSql.read.load('/tmp/asistencia.csv',
                           format="csv",sep=',',
                           inferSchema='true',
                           header='true')
Evaluacion = ContextoSql.read.load('/tmp/evaluacion.csv',
                           format="csv",sep=',',
                           inferSchema='true',
                           header='true')

In [6]:
Asistencia.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo al que perteneces:: string (nullable = true)
 |-- Equipo que va a exponer:: string (nullable = true)



In [7]:
Evaluacion.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo que vas a evaluar:: string (nullable = true)
 |-- Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?: integer (nullable = true)
 |-- Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?: integer (nullable = true)
 |-- Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?: integer (nullable = true)
 |-- Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?: integer (nullable = true)
 |-- Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?: integer (nullable = true)
 |-- Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.: integer (nullable = true)
 |-- Tracción: El equipo responde adecuadamente si cuenta con clientes que demues

In [8]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

Evaluacion = Evaluacion.filter(Evaluacion["Nombre de usuario"].contains("@"))
Evaluacion = Evaluacion.withColumn('Nombre de usuario',trim(Evaluacion['Nombre de usuario'])) ## trim 
Evaluacion.select('Nombre de usuario').distinct().orderBy('Nombre de usuario',ascending=True).show(10000)

+--------------------+
|   Nombre de usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
|dgerenal@unal.edu.co|
|dgiraldolo@unal.e...|
|dguardia@unal.edu.co|
|dosoriom@unal.edu.co|
| eapenad@unal.edu.co|
|elondonoc@unal.ed...|
|emflorezb@unal.ed...|
|emlopezr@unal.edu.co|
|eporrasm@unal.edu.co|
|esgarciac@unal.ed...|
|favasquezj@unal.e...|
|fguerrerot@unal.e...|
|fmiranda@unal.edu.co|
|framirezf@unal.ed...|
|hriveraa@unal.edu.co|
|jcardonaso@unal.e...|
|jcorreapa@unal.ed...|
|jdroldano@unal.ed...|
|jfernandezmo@unal...|
|jgutierrezlo@unal...|
|jhcordoba@

In [9]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

Asistencia = Asistencia.withColumn("Marca temporal",(to_timestamp(Asistencia["Marca temporal"], 'yyyy/MM/dd HH:mm:ss')))

Asistencia.groupBy("Nombre de usuario").agg(F.count('Nombre de usuario').alias("Número de asistencias"), 
                                            F.max("Marca temporal").alias("Última asistencia"),
                                            F.min("Marca temporal").alias("Primera asistencia")).orderBy(["Número de asistencias","Nombre de usuario"],ascending=[False,True]).show(3000)

+--------------------+---------------------+-------------------+-------------------+
|   Nombre de usuario|Número de asistencias|  Última asistencia| Primera asistencia|
+--------------------+---------------------+-------------------+-------------------+
|anapariciom@unal....|                   10|2021-04-13 09:38:51|2021-04-08 08:11:49|
|favasquezj@unal.e...|                   10|2021-04-13 09:41:00|2021-04-08 08:10:59|
|jovillarrealm@una...|                   10|2021-04-13 09:42:55|2021-04-08 08:22:36|
|samolinap@unal.ed...|                   10|2021-04-13 09:37:26|2021-04-08 08:11:43|
| dadazam@unal.edu.co|                    9|2021-04-13 09:41:14|2021-04-08 08:13:18|
|dgarciabl@unal.ed...|                    9|2021-04-13 09:41:04|2021-04-08 08:09:39|
|esgarciac@unal.ed...|                    9|2021-04-13 09:39:45|2021-04-08 08:24:10|
|jcardonaso@unal.e...|                    9|2021-04-13 09:44:24|2021-04-08 08:10:54|
|jlopezpe@unal.edu.co|                    9|2021-04-13 09:09:32|2

In [11]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

grupo_asistencias = Asistencia.groupBy("Nombre de usuario").agg(F.count('Nombre de usuario').alias("Número de asistencias"), 
                                            F.max("Marca temporal").alias("Última asistencia"),
                                            F.min("Marca temporal").alias("Primera asistencia")).orderBy(["Número de asistencias","Nombre de usuario"],ascending=[False,True])

grupo_asistencias.withColumn("Número de asistencias",
                             grupo_asistencias["Número de asistencias"].cast(IntegerType())).where(
                             grupo_asistencias['Número de asistencias']<=2).show(1000)

+--------------------+---------------------+-------------------+-------------------+
|   Nombre de usuario|Número de asistencias|  Última asistencia| Primera asistencia|
+--------------------+---------------------+-------------------+-------------------+
|jdroldano@unal.ed...|                    1|2021-04-08 08:20:16|2021-04-08 08:20:16|
+--------------------+---------------------+-------------------+-------------------+



In [13]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...


diferentes_estudiantes = Evaluacion.select('Nombre de usuario').distinct().orderBy('Nombre de usuario',ascending=True)
diferentes_estudiantes.join(grupo_asistencias,on=["Nombre de usuario"],how = 'left_anti').show(10)


+-----------------+
|Nombre de usuario|
+-----------------+
+-----------------+



In [15]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

Asistencia_fixed = Asistencia.withColumn("Equipo al que perteneces:", regexp_replace("Equipo al que perteneces:", 'Pertenezco a:', ''))
Asistencia_fixed = Asistencia_fixed.withColumn("Equipo que va a exponer:", regexp_replace("Equipo que va a exponer:", 'Expone: ',""))

Asistencia_fixed.groupby("Equipo al que perteneces:").agg(concat_ws(", ",collect_list("Nombre de usuario")).alias("Integrantes")).toPandas()

,Equipo al que perteneces:,Integrantes
0,Lo tengo y te lo presto.,"dgarciabl@unal.edu.co, jmunozhe@unal.edu.co, d..."
1,Gestión de Residencias y Habitaciones Univers...,"auarbelaeza@unal.edu.co, seagudeloo@unal.edu.c..."
2,Gestión de programas curriculares,"jucardonaa@unal.edu.co, jmonsalveb@unal.edu.co..."
3,Mi Sanandresito,"fmiranda@unal.edu.co, eporrasm@unal.edu.co, af..."
4,Servicios a domicilio,"jppinedal@unal.edu.co, jmunozbo@unal.edu.co, a..."
5,Administración de Inmuebles,"lumarinb@unal.edu.co, framirezf@unal.edu.co, s..."
6,Te lo cambio.,"dgerenal@unal.edu.co, nvalenciat@unal.edu.co, ..."
7,Gestión de Menús Ejecutivos,"jcardonaso@unal.edu.co, favasquezj@unal.edu.co..."


In [18]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

Asistencia_fixed_2 = Asistencia_fixed.withColumn("Equipo que va a exponer:",trim(Asistencia_fixed["Equipo que va a exponer:"])) ## trim 
Asistencia_fixed_2 = Asistencia_fixed_2.withColumn("Equipo al que perteneces:",trim(Asistencia_fixed_2["Equipo al que perteneces:"])) ## trim 

Asistencia_fixed_2.where(Asistencia_fixed_2["Equipo que va a exponer:"] != 
                         Asistencia_fixed_2["Equipo al que perteneces:"]).groupBy("Equipo que va a exponer:").agg(
                         F.count("Nombre de usuario").alias("Asistentes")).orderBy("Asistentes",ascending=False).show(1000)

+------------------------+----------+
|Equipo que va a exponer:|Asistentes|
+------------------------+----------+
|    Servicios a domic...|        76|
|           Te lo cambio.|        75|
|    Lo tengo y te lo ...|        75|
|    Gestión de progra...|        74|
|    Gestión de Menús ...|        72|
|         Mi Sanandresito|        72|
|    Gestión de Reside...|        72|
|    Administración de...|        70|
|    Deserción de estu...|         3|
+------------------------+----------+



In [22]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

Evaluacion2 = Evaluacion.withColumn('Equipo que vas a evaluar:', trim(Evaluacion['Equipo que vas a evaluar:']))
Evaluacion2 = Evaluacion2.select('Nombre de usuario', 'Equipo que vas a evaluar:').distinct()


Equipos = Evaluacion2.join(Asistencia_fixed_2.select('Nombre de usuario', 'Equipo al que perteneces:').distinct(), 
                           on = ['Nombre de usuario'], how = 'left')
Equipos.filter(Equipos['Equipo que vas a evaluar:'] == Equipos['Equipo al que perteneces:']).orderBy('Nombre de usuario').show(1000)

+--------------------+-------------------------+-------------------------+
|   Nombre de usuario|Equipo que vas a evaluar:|Equipo al que perteneces:|
+--------------------+-------------------------+-------------------------+
|anapariciom@unal....|     Servicios a domic...|     Servicios a domic...|
|daestradam@unal.e...|     Lo tengo y te lo ...|     Lo tengo y te lo ...|
|hriveraa@unal.edu.co|     Gestión de Menús ...|     Gestión de Menús ...|
|jhcordoba@unal.ed...|     Administración de...|     Administración de...|
| jruedat@unal.edu.co|     Servicios a domic...|     Servicios a domic...|
| kmolano@unal.edu.co|     Administración de...|     Administración de...|
|  lmayar@unal.edu.co|     Gestión de progra...|     Gestión de progra...|
|mcarvajalsa@unal....|     Gestión de Reside...|     Gestión de Reside...|
+--------------------+-------------------------+-------------------------+



In [73]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

Evaluacion2 = Evaluacion.withColumn('Equipo que vas a evaluar:', trim(Evaluacion['Equipo que vas a evaluar:']))


Evaluacion2 = Evaluacion2.select('Nombre de usuario', 'Equipo que vas a evaluar:', 
                                Evaluacion2['Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?'].alias('Introducción'),
                                Evaluacion2['Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?'].alias('Equipo'),
                                Evaluacion2['Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?'].alias('Problema'),
                                Evaluacion2['Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?'].alias('Ventajas'),
                                Evaluacion2['Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?'].alias('Solución'),
                                Evaluacion2["`Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.`"].alias('Producto'),
                                Evaluacion2["`Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.`"].alias('Tracción'),
                                Evaluacion2["`Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.`"].alias('Mercado') ,
                                Evaluacion2["`Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?`"].alias('Competencia'),
                                Evaluacion2["`Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? `"].alias('Modelo de negocio'),
                                Evaluacion2["`Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?`"].alias('Inversión'),
                                Evaluacion2["`Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.`"].alias('Contacto'),
                                Evaluacion2["`Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?`"].alias('Exposicion1'),
                                Evaluacion2["`Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?`"].alias('Exposicion2'),
                                Evaluacion2["`Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.`"].alias('Exposicion3'))

Equipos = Evaluacion2.join(Asistencia_fixed_2.select('Nombre de usuario', 'Equipo al que perteneces:').distinct(), on = ['Nombre de usuario'], how = 'left')
evaluaciones = Equipos.filter(Equipos['Equipo que vas a evaluar:'] != Equipos['Equipo al que perteneces:'])

promedios = evaluaciones.groupBy('Equipo que vas a evaluar:').agg(F.avg('Introducción'), F.avg('Equipo'), F.avg('Problema'), 
                                                                  F.avg('Ventajas'), F.avg('Solución'), F.avg('Producto'), 
                                                                  F.avg('Tracción'), F.avg('Mercado'), F.avg('Competencia'), 
                                                                  F.avg('Modelo de negocio'), F.avg('Inversión'), F.avg('Contacto'), 
                                                                  F.avg('Exposicion1'), F.avg('Exposicion2'), F.avg('Exposicion3'))

NotaFinal = promedios.select('*', ((promedios["`avg(Introducción)`"] + promedios["`avg(Equipo)`"] + promedios["`avg(Problema)`"] + promedios["`avg(Ventajas)`"] + promedios["`avg(Solución)`"] + promedios["`avg(Producto)`"] + promedios["`avg(Tracción)`"] + 
                             promedios["`avg(Mercado)`"] + promedios["`avg(Competencia)`"] + promedios["`avg(Modelo de negocio)`"] + promedios["`avg(Inversión)`"] + promedios["`avg(Contacto)`"] + promedios["`avg(Exposicion1)`"] + promedios["`avg(Exposicion2)`"] + 
                             promedios["`avg(Exposicion3)`"])/15).alias('Nota final'))

NotaFinal.toPandas()

,Equipo que vas a evaluar:,avg(Introducción),avg(Equipo),avg(Problema),avg(Ventajas),avg(Solución),avg(Producto),avg(Tracción),avg(Mercado),avg(Competencia),avg(Modelo de negocio),avg(Inversión),avg(Contacto),avg(Exposicion1),avg(Exposicion2),avg(Exposicion3),Nota final
0,Servicios a domicilio,4.486486,4.635135,4.540541,4.270270,4.513514,4.594595,4.135135,4.148649,4.351351,4.297297,4.162162,4.648649,4.567568,4.567568,4.608108,4.435135
1,Lo tengo y te lo presto.,4.465753,4.589041,4.273973,4.438356,4.410959,4.712329,4.356164,4.273973,4.493151,4.493151,4.356164,4.602740,4.602740,4.602740,4.630137,4.486758
2,Gestión de Residencias y Habitaciones Universi...,4.555556,4.430556,4.652778,4.458333,4.472222,4.125000,4.527778,4.680556,4.402778,4.513889,4.333333,4.638889,4.527778,4.527778,4.388889,4.482407
3,Te lo cambio.,4.328947,4.013158,4.184211,4.000000,4.421053,4.828947,4.052632,4.052632,4.210526,4.315789,3.644737,4.500000,4.486842,4.486842,4.552632,4.271930
4,Administración de Inmuebles,4.111111,4.402778,3.972222,3.930556,4.152778,4.444444,4.041667,4.416667,3.972222,4.416667,4.111111,4.152778,4.222222,4.222222,4.138889,4.180556
5,Deserción de estudiantes,4.500000,5.000000,5.000000,5.000000,4.500000,5.000000,5.000000,4.500000,5.000000,4.500000,4.500000,5.000000,4.500000,4.500000,5.000000,4.766667
6,Gestión de Menús Ejecutivos,4.337838,3.986486,4.567568,4.324324,4.364865,4.040541,4.202703,4.243243,4.297297,4.635135,4.297297,4.500000,4.445946,4.445946,4.743243,4.362162
7,Gestión de programas curriculares,4.386667,4.400000,4.546667,3.920000,4.306667,4.466667,4.386667,4.506667,4.240000,4.293333,4.506667,4.386667,4.480000,4.480000,4.626667,4.395556
8,Mi Sanandresito,4.194805,3.935065,4.285714,4.012987,4.324675,4.298701,4.233766,4.415584,3.545455,4.376623,4.363636,4.623377,4.168831,4.168831,4.142857,4.206061


In [105]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

def maximo(colname):
    NotaFinal.where(NotaFinal[colname] == NotaFinal.agg({colname: "max"}).collect()[0][0]).select('Equipo que vas a evaluar:',colname).show()

for col in NotaFinal.columns[1:]:
    maximo(col)

+-------------------------+-----------------+
|Equipo que vas a evaluar:|avg(Introducción)|
+-------------------------+-----------------+
|     Gestión de Reside...|4.555555555555555|
+-------------------------+-----------------+

+-------------------------+-----------+
|Equipo que vas a evaluar:|avg(Equipo)|
+-------------------------+-----------+
|     Deserción de estu...|        5.0|
+-------------------------+-----------+

+-------------------------+-------------+
|Equipo que vas a evaluar:|avg(Problema)|
+-------------------------+-------------+
|     Deserción de estu...|          5.0|
+-------------------------+-------------+

+-------------------------+-------------+
|Equipo que vas a evaluar:|avg(Ventajas)|
+-------------------------+-------------+
|     Deserción de estu...|          5.0|
+-------------------------+-------------+

+-------------------------+-----------------+
|Equipo que vas a evaluar:|    avg(Solución)|
+-------------------------+-----------------+
|     Se

In [106]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...


def minimo(colname):
    NotaFinal.where(NotaFinal[colname] == NotaFinal.agg({colname: "min"}).collect()[0][0]).select('Equipo que vas a evaluar:',colname).show()

for col in NotaFinal.columns[1:]:
    minimo(col)

+-------------------------+-----------------+
|Equipo que vas a evaluar:|avg(Introducción)|
+-------------------------+-----------------+
|     Administración de...|4.111111111111111|
+-------------------------+-----------------+

+-------------------------+-----------------+
|Equipo que vas a evaluar:|      avg(Equipo)|
+-------------------------+-----------------+
|          Mi Sanandresito|3.935064935064935|
+-------------------------+-----------------+

+-------------------------+------------------+
|Equipo que vas a evaluar:|     avg(Problema)|
+-------------------------+------------------+
|     Administración de...|3.9722222222222223|
+-------------------------+------------------+

+-------------------------+-------------+
|Equipo que vas a evaluar:|avg(Ventajas)|
+-------------------------+-------------+
|     Gestión de progra...|         3.92|
+-------------------------+-------------+

+-------------------------+-----------------+
|Equipo que vas a evaluar:|    avg(Solución)|

In [125]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

Asistencia_fixed_3 = Asistencia_fixed.withColumn('Equipo al que perteneces:', trim(Asistencia_fixed['Equipo al que perteneces:']))

Asistencia_fixed_3.select('Nombre de usuario', 
                        'Equipo al que perteneces:').distinct().join(NotaFinal, 
                                                                     on = (Asistencia_fixed_3['Equipo al que perteneces:'] == NotaFinal['Equipo que vas a evaluar:']) , 
                                                                     how='left').select('Nombre de usuario', 'Equipo al que perteneces:', 'Nota final').show()

+--------------------+-------------------------+-----------------+
|   Nombre de usuario|Equipo al que perteneces:|       Nota final|
+--------------------+-------------------------+-----------------+
|sebermudezg@unal....|     Servicios a domic...|4.435135135135135|
|jmunozbo@unal.edu.co|     Servicios a domic...|4.435135135135135|
|anapariciom@unal....|     Servicios a domic...|4.435135135135135|
| bocampo@unal.edu.co|     Servicios a domic...|4.435135135135135|
|sacastrot@unal.ed...|     Servicios a domic...|4.435135135135135|
|jppinedal@unal.ed...|     Servicios a domic...|4.435135135135135|
|dosoriom@unal.edu.co|     Servicios a domic...|4.435135135135135|
|jdroldano@unal.ed...|     Servicios a domic...|4.435135135135135|
|cjfunezg@unal.edu.co|     Servicios a domic...|4.435135135135135|
| jruedat@unal.edu.co|     Servicios a domic...|4.435135135135135|
|jfernandezmo@unal...|     Lo tengo y te lo ...|4.486757990867579|
|jcorreapa@unal.ed...|     Lo tengo y te lo ...|4.486757990867